In [1]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
import numpy as np
import tflearn
import tensorflow
import random
import json
import pickle
from tensorflow.python.framework import ops

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2
INFO:tensorflow:Disabling eager execution
INFO:tensorflow:Disabling v2 tensorshape
Instructions for updating:
non-resource variables are not supported in the long term
INFO:tensorflow:Disabling resource variables
INFO:tensorflow:Disabling tensor equality
INFO:tensorflow:Disabling control flow v2
curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [2]:
# nltk.download('punkt')
stemmer = LancasterStemmer()

In [3]:
with open('intents.json') as file:
    data =  json.load(file)

In [4]:

# have each word in pattern
words = []
# tags
labels = []
# patterns
doc_x = []
doc_y = []

In [5]:
for intent in data['intents']:
    for pattern in intent['patterns']:
        # saperates the word from string
        wrds = nltk.word_tokenize(pattern)
        words.extend(wrds)
        doc_x.append(pattern)
        doc_y.append(intent['tag'])

        if intent['tag'] not in labels:
            labels.append(intent['tag'])

In [6]:
print("Words: ",words)
print("Labels",labels)
print("Doc_x",doc_x)
print("Doc_y",doc_y)

Words:  ['Hi', 'How', 'are', 'you', 'Is', 'anyone', 'there', '?', 'Hello', 'Good', 'day', 'Whats', 'up', 'cya', 'See', 'you', 'later', 'Goodbye', 'I', 'am', 'Leaving', 'Have', 'a', 'Good', 'day', 'how', 'old', 'how', 'old', 'is', 'tim', 'what', 'is', 'your', 'age', 'how', 'old', 'are', 'you', 'age', '?', 'what', 'is', 'your', 'name', 'what', 'should', 'I', 'call', 'you', 'whats', 'your', 'name', '?', 'Id', 'like', 'to', 'buy', 'something', 'whats', 'on', 'the', 'menu', 'what', 'do', 'you', 'reccommend', '?', 'could', 'i', 'get', 'something', 'to', 'eat', 'when', 'are', 'you', 'guys', 'open', 'what', 'are', 'your', 'hours', 'hours', 'of', 'operation']
Labels ['greeting', 'goodbye', 'age', 'name', 'shop', 'hours']
Doc_x ['Hi', 'How are you', 'Is anyone there?', 'Hello', 'Good day', 'Whats up', 'cya', 'See you later', 'Goodbye', 'I am Leaving', 'Have a Good day', 'how old', 'how old is tim', 'what is your age', 'how old are you', 'age?', 'what is your name', 'what should I call you', 'wha

In [7]:
words = [stemmer.stem(w.lower()) for w in words]

In [8]:
print(words)

['hi', 'how', 'ar', 'you', 'is', 'anyon', 'ther', '?', 'hello', 'good', 'day', 'what', 'up', 'cya', 'see', 'you', 'lat', 'goodby', 'i', 'am', 'leav', 'hav', 'a', 'good', 'day', 'how', 'old', 'how', 'old', 'is', 'tim', 'what', 'is', 'yo', 'ag', 'how', 'old', 'ar', 'you', 'ag', '?', 'what', 'is', 'yo', 'nam', 'what', 'should', 'i', 'cal', 'you', 'what', 'yo', 'nam', '?', 'id', 'lik', 'to', 'buy', 'someth', 'what', 'on', 'the', 'menu', 'what', 'do', 'you', 'reccommend', '?', 'could', 'i', 'get', 'someth', 'to', 'eat', 'when', 'ar', 'you', 'guy', 'op', 'what', 'ar', 'yo', 'hour', 'hour', 'of', 'op']


In [9]:
# set makes suere there is no duplicate words
# list makes the set into list
# and then its been sorted
words = sorted(list(set(words)))
# words

In [10]:
# words

In [11]:
labels = sorted(labels)
labels

['age', 'goodbye', 'greeting', 'hours', 'name', 'shop']

In [12]:
# te

In [13]:
try:
    with open("data.pickle", "rb") as f:
        words, labels, training, output = pickle.load(f)

except:
    # have each word in pattern
    words = []
    # tags
    labels = []
    # patterns
    doc_x = []
    doc_y = []

    for intent in data['intents']:
        for pattern in intent['patterns']:
            # saperates the word from string
            wrds = nltk.word_tokenize(pattern)
            words.extend(wrds)
            doc_x.append(wrds)
            doc_y.append(intent['tag'])

            if intent['tag'] not in labels:
                labels.append(intent['tag'])


    # arranging data
    words = [stemmer.stem(w.lower()) for w in words]
    words = sorted(list(set(words)))

    labels = sorted(labels)

    training = []
    output = []

    out_empty = [0 for _ in range(len(labels))]

    # saperating data for training and output
    for x , doc in enumerate(doc_x):
        bag = []
        wrds = [stemmer.stem(w) for w in doc]
        
        for w in words:
            if w in wrds:
                bag.append(1)
            else:
                bag.append(0)
        
        output_row = out_empty[:]
        output_row[labels.index(doc_y[x])] = 1
        
    training.append(bag)
    output.append(output_row)  
    
    with open("data.pickle", "wb") as f:
        pickle.dump((words, labels, training, output), f)


training = np.array(training)
output = np.array(output)

In [14]:
ops.reset_default_graph()

# nuron connection

# 47 * 8 * 8 * 6
net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation='softmax')
net = tflearn.regression(net)

# training model

model = tflearn.DNN(net)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [15]:
try:
    model.load("model.tflearn")

except:
    model.fit(training, output, n_epoch = 1000, batch_size=8, show_metric=True)
    model.save('model.tflearn')

INFO:tensorflow:Restoring parameters from P:\Python Chatbot\model.tflearn


In [16]:
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]

    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1


    return np.array(bag)


In [17]:
def chat():
    print("Start talking with the bot! (type quit to stop)")
    while True:
        inp = input("You: ")
        if inp.lower() == "quit":
            print("Thanks for your good time")
            break
            
        results = model.predict([bag_of_words(inp, words)])
        result_index = np.argmax(results)
        tag = labels[result_index]
        print(results)
        print(result_index)
        print(tag)

In [ ]:
chat()

Start talking with the bot! (type quit to stop)
You: hello
[[0.29938087 0.21807158 0.37696758 0.01044051 0.0016073  0.09353221]]
2
greeting
You: hiii
[[0.3002157  0.2184573  0.37952858 0.00967375 0.00140404 0.0907206 ]]
2
greeting
You: hours
[[0.2962151  0.22502838 0.37983972 0.00967382 0.00131939 0.08792365]]
2
greeting
You: goodbye
[[0.3018167  0.21400939 0.37657958 0.01046091 0.00167723 0.09545609]]
2
greeting
You: how old
[[0.30289423 0.21451217 0.37987715 0.00949332 0.0014119  0.09181124]]
2
greeting
